<a href="https://colab.research.google.com/github/Hameon4/Tensorflow-2.0-Practice/blob/main/LAB2_2_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/Hameon4/Exfiltration-dataset/main/exfil_monogram.csv

In [ ]:
!pip install ctgan
!pip install table_evaluator

In [3]:
import pandas as pd 
data = pd.read_csv('/content/exfil_monogram.csv')
data = data.drop(columns = ['Label'])

In [4]:
# Defining a list to pass to the model
categorical_features = [data.columns[i] for i in range(len(data.columns))]

In [5]:
# Train the model 
from ctgan import CTGANSynthesizer

ctgan = CTGANSynthesizer(verbose=True)
ctgan.fit(data, categorical_features, epochs = 20)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


Epoch 1, Loss G:  5.5117,Loss D: -7.0452
Epoch 2, Loss G:  3.1563,Loss D: -15.5744
Epoch 3, Loss G: -3.4036,Loss D: -14.0647
Epoch 4, Loss G: -7.6994,Loss D: -10.2549
Epoch 5, Loss G: -8.8790,Loss D: -5.3425
Epoch 6, Loss G: -9.1807,Loss D: -3.5605
Epoch 7, Loss G: -9.2275,Loss D: -1.8244
Epoch 8, Loss G: -7.5351,Loss D:  0.0494
Epoch 9, Loss G: -5.1148,Loss D:  0.6289
Epoch 10, Loss G: -4.7322,Loss D: -0.2491
Epoch 11, Loss G: -2.8186,Loss D: -0.5558
Epoch 12, Loss G: -4.3923,Loss D:  0.5691
Epoch 13, Loss G: -2.8687,Loss D: -0.4818
Epoch 14, Loss G: -3.3528,Loss D: -0.9775
Epoch 15, Loss G: -3.3816,Loss D: -0.1535
Epoch 16, Loss G: -4.6985,Loss D:  0.3650
Epoch 17, Loss G: -5.1676,Loss D:  0.1369
Epoch 18, Loss G: -4.5494,Loss D:  1.1003
Epoch 19, Loss G: -4.1428,Loss D:  0.4003
Epoch 20, Loss G: -2.7049,Loss D:  0.1215
